# Sudo code:
### Process to obtain the data:
1. Get the number of posts.
    * define city name.
    * create url with get_url function
    * requests.get(url) to get response
    * use BeautifuSoup to get entire posts text
    * Find total number of posts then return 
2. Use the total number of posts to get each pages first post number (119 posts per page)
    * List name pages e.g. [0, 120, 240, 360, ........]
3. Scrape the information I need and store in the list
    * loop each pages list, it opens the page starting with the posts number
    * Create function to obtain informations.
        This will scrape each post, then store the information. 
4. Creat DataFrame with the information I scraped.
    * Store in the list of empty DataFrames 
    * Save as csv file


#### Sub_city List:
* sub_city_van_area: 
        * 'van'- Vancouver
        * 'bnc' - Burnaby/NewWest
        * 'rds' - Delta/Surrey/Langley
        * 'nvn' - North Shore
        * 'rch' - Richmond
        * 'pml' - Tricities/Pittmedow/Maple Ridge
* sub_city_tor_area: 
        * 'tor' - City of Toronto
        * 'bra' - Brampton
        * 'drh' - Durham Region
        * 'mss' - Mississauga
        * 'oak' - Oakville
        * 'yrk' - York Region

In [17]:
from requests import get
from bs4 import BeautifulSoup
from time import sleep
import numpy as np
import pandas as pd

def get_search_number(url, city, sub_city):
    response = get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    posts = soup.find_all('li', class_='result-row')
    results_num = soup.find('div', class_='search-legend')
    results_total = int(results_num.find('span', class_='totalcount').text)
    print(f"Total number of search result in {city} - {sub_city}: {results_total}")
    return results_total

In [11]:
class Url:
    def __init__(self, city_name, sub_city=None, num_post=0):
        self.city_name = city_name
        self.sub_city = sub_city
        self.num_post = num_post
        self.url = self.get_url()
        
    def get_url(self):
        if self.city_name == 'Abbotsford':
            url = f'https://{self.city_name.lower()}.craigslist.org/search/apa?s={self.num_post}sort=date&bundleDuplicates=1&min_price=99max_price=&availabilityMode=0&sale_date=all+dates'
        else:
            url = f'https://{self.city_name.lower()}.craigslist.org/search/{self.sub_city}/apa?s={self.num_post}sort=date&bundleDuplicates=1&min_price=99max_price=&availabilityMode=0&sale_date=all+dates'
        return url   

In [26]:
%%time
city_names = ["Vancouver", "Toronto"]
dfs = []
for city in city_names:
    if city == 'Vancouver':
        sub_city_names = ['van', 'bnc', 'rds', 'nvn', 'rch', 'pml']
    elif city == 'Toronto':
        sub_city_names = ['tor', 'bra', 'drh', 'mss', 'oak', 'yrk']
    for sub_city in sub_city_names:
        url = Url(city, sub_city)
        url_ = url.url
        results_total = get_search_number(url_, city, sub_city)
        
        pages = np.arange(0, results_total+1, 120)
        
        iterations = 0

        post_date = []
        post_cities = []
        num_bedroom = []
        sqfts = []
        prices = []
        post_titles = []
        post_links = []

        for page in pages:
     
            # get request
            url = Url(city_name=city, sub_city=sub_city, num_post=page)
            url_ = url.url
            response = get(url_)
    
            sleep(1)
    
            # throw warning for status code that are not 200
            if response.status_code != 200:
                warn(f'Request: {requests}; Status code: {response.status_code}')
        
            soup = BeautifulSoup(response.text, 'html.parser')
    
            posts = soup.find_all('li', class_='result-row')
    
    
            # Extract data 
            for post in posts:
        
                if post.find('span', class_ = 'result-hood') is not None:
            
                    # Posting date
                    post_datetime = post.find('time', class_='result-date')['datetime']
                    post_date.append(post_datetime)
            
                    # Neighbourhoods
                    post_city = post.find('span', class_='result-hood').text.strip('( )')
                    post_cities.append(post_city)
            
                    # title text
                    post_title = post.find('a', class_='result-title hdrlnk')
                    post_title_text = post_title.text
                    post_titles.append(post_title_text)
            
                    # Post price in integer
                    price = int(post.find('span', class_='result-price').text.split('$')[1].replace(',', ''))
                    prices.append(price)
            
                    # Post link
                    post_link = post_title['href']
                    post_links.append(post_link)
            
                    if post.find('span', class_ = 'housing') is not None:
                
                        #if the first element is accidentally square footage
                        if 'ft2' in post.find('span', class_ = 'housing').text.split()[0]:
                    
                            #make bedroom nan
                            bedroom_count = np.nan
                            num_bedroom.append(bedroom_count)
                    
                            #make sqft the first element
                            sqft = int(post.find('span', class_ = 'housing').text.split()[0][:-3])
                            sqfts.append(sqft)
                    
                        #if the length of the housing details element is more than 2
                        elif len(post.find('span', class_ = 'housing').text.split()) > 2:
                    
                            #therefore element 0 will be bedroom count
                            bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                            num_bedroom.append(bedroom_count)
                    
                            #and sqft will be number 3, so set these here and append
                            sqft = int(post.find('span', class_ = 'housing').text.split()[2][:-3])
                            sqfts.append(sqft)
                    
                        #if there is num bedrooms but no sqft
                        elif len(post.find('span', class_ = 'housing').text.split()) == 2:
                    
                            #therefore element 0 will be bedroom count
                            bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                            num_bedroom.append(bedroom_count)
                    
                            #and sqft will be number 3, so set these here and append
                            sqft = np.nan
                            sqfts.append(sqft)                    
                
                        else:
                            bedroom_count = np.nan
                            num_bedroom.append(bedroom_count)
                
                            sqft = np.nan
                            sqfts.append(sqft)
                
                    #if none of those conditions catch, make bedroom nan, this won't be needed    
                    else:
                        bedroom_count = np.nan
                        num_bedroom.append(bedroom_count)
                
                        sqft = np.nan
                        sqfts.append(sqft)
                
                iterations += 1
        print(f"Total Number of post scraped: {iterations}")
        
        # Store in Pandas DataFrame then append to the list of DataFrame
        df = pd.DataFrame({'Post Datetime': post_date,
                           'Post Title': post_titles,
                           'Post URL': post_links,
                          'Neighborhood': post_cities,
                          'Bedroom': num_bedroom,
                          'SQFT': sqfts,
                          'Price': prices})
        
        dfs.append(df)
        
        # Save DataFrame in CSV file. 
        df.to_csv(f'C:\\Users\\Masa\\Desktop\\Chilliwack Real Estate\\Data\\{city}_{sub_city}_rental_price_data.csv',
                 header=False, mode='a')

Total number of search result in Vancouver - van: 3000
Total Number of post scraped: 3052
Total number of search result in Vancouver - bnc: 1296
Total Number of post scraped: 1318
Total number of search result in Vancouver - rds: 1810
Total Number of post scraped: 1845
Total number of search result in Vancouver - nvn: 743
Total Number of post scraped: 774
Total number of search result in Vancouver - rch: 368
Total Number of post scraped: 383
Total number of search result in Vancouver - pml: 847
Total Number of post scraped: 852
Total number of search result in Toronto - tor: 3000
Total Number of post scraped: 3033
Total number of search result in Toronto - bra: 55
Total Number of post scraped: 55
Total number of search result in Toronto - drh: 46
Total Number of post scraped: 46
Total number of search result in Toronto - mss: 241
Total Number of post scraped: 241
Total number of search result in Toronto - oak: 9
Total Number of post scraped: 61
Total number of search result in Toronto 

In [27]:
len(dfs)

12

In [28]:
for df in dfs:
    print(len(df))

3052
1318
1845
774
383
852
3033
55
46
241
18
108


In [33]:
# As you can tell the number of scraped data is exceeding the number of the post, so I remove duplicate. 
df = pd.read_csv('C:\\Users\\Masa\\Desktop\\Chilliwack Real Estate\\Data\\Vancouver_bnc_rental_price_data.csv').drop('Unnamed: 0', axis=1)
df.drop_duplicates(inplace=True)
len(df)

1293

In [36]:
df.head()

,Post Datetime,Post Title,Post URL,Neighborhood,Bedroom,SQFT,Price
0,2022-12-10 17:16,Spacious 2 bedroom Apartment with in suite lau...,https://vancouver.craigslist.org/bnc/apa/d/new...,"527 Carnarvon Street, New Westminster, BC",2.0,716.0,2745
1,2022-12-10 17:13,Brentwood Burnaby Resort Style One Bedroom Condo,https://vancouver.craigslist.org/bnc/apa/d/bur...,4720 Lougheed Hwy,1.0,535.0,2400
2,2022-12-10 17:13,Two Bedroom Apartment at SFU Burnaby,https://vancouver.craigslist.org/bnc/apa/d/two...,9060 University Crescent,2.0,927.0,2600
3,2022-12-10 17:00,1 bedroom modern living at The Columbia New We...,https://vancouver.craigslist.org/bnc/apa/d/new...,Brewery district,1.0,560.0,2200
4,2022-12-10 16:59,Rare huge FULLY FURNISHED 1 bedroom at Modello...,https://vancouver.craigslist.org/bnc/apa/d/bur...,Metrotown,1.0,760.0,2800


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1293 entries, 0 to 1317
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Post Datetime  1293 non-null   object 
 1   Post Title     1293 non-null   object 
 2   Post URL       1293 non-null   object 
 3   Neighborhood   1293 non-null   object 
 4   Bedroom        1257 non-null   float64
 5   SQFT           1090 non-null   float64
 6   Price          1293 non-null   int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 80.8+ KB
